#Web and Text Analytics 
### This Code has been run on colab. This explains commands with "!" in front and the drive import 
 

   © Matthias Pirlet, François Lievens, Julien Hubar


In [3]:
#import drive 
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!git clone https://github.com/empenguinxh/PyWME

Cloning into 'PyWME'...
remote: Enumerating objects: 22, done.
remote: Counting objects: 100% (22/22), done.
remote: Compressing objects: 100% (17/17), done.
remote: Total 22 (delta 8), reused 14 (delta 4), pack-reused 0
Unpacking objects: 100% (22/22), done.


/!\ if you want to use this, you need to modify the path in the file wme.py by "/content/drive/MyDrive/data/"

In [ ]:
# Create the numpy array with the document embeddings
!python /content/PyWME/wme.py /content/drive/MyDrive/data/data50k_tweet.txt --R 256

experiment configuration: Namespace(D_max=6, D_min=1, R=256, corpus_fp='/content/drive/MyDrive/data/data50k_tweet.txt', exp_id='exp_0', gamma=1.0, no_cache=False, nuw_max=500, wv_name='word2vec-google-news-300')
... loading word vectors: word2vec-google-news-300 ...
tcmalloc: large alloc 3600007168 bytes == 0x560191718000 @  0x7fdb69703001 0x7fdb64fe554f 0x7fdb65035b58 0x7fdb65039b17 0x7fdb650d8203 0x56018f096544 0x56018f096240 0x56018f10a627 0x56018f104ced 0x56018f097bda 0x56018f106737 0x56018f1049ee 0x56018f097bda 0x56018f106737 0x56018f097afa 0x56018f109d00 0x56018f1049ee 0x56018f097bda 0x56018f109d00 0x56018f104ced 0x56018efd6e2b 0x56018f106fe4 0x56018f1049ee 0x56018f1046f3 0x56018f1ce4c2 0x56018f1ce83d 0x56018f1ce6e6 0x56018f1a6163 0x56018f1a5e0c 0x7fdb684ebbf7 0x56018f1a5cea
... loading corpus: /content/drive/MyDrive/data/data50k_tweet.txt ...
... 50000 docs in total ...
... preparing random docs ...
... preparing docs ...
... computing WME vectors...
... load cache for all docs 

In [4]:
import numpy as np
#wme_mat is numpy array with the number of line = number of documents
wme_mat = np.load('/content/drive/MyDrive/data/data50k_tweet_wme_exp_0.npy')

In [5]:
print(wme_mat[420])
print("Number of documents :", wme_mat.shape[0])
print("Size of the embedding :", wme_mat.shape[1])

[0.00231428 0.00239608 0.00255265 0.00249867 0.00257175 0.00246384
 0.00252394 0.00243477 0.00240978 0.00254249 0.00241538 0.00236164
 0.00262419 0.00253007 0.00252735 0.00250423 0.00244535 0.00255964
 0.00247979 0.00255023 0.00237222 0.00253147 0.00247082 0.00255835
 0.00245778 0.00236027 0.00248565 0.00248544 0.00244721 0.00254247
 0.00227787 0.00253978 0.00252791 0.00250085 0.00247678 0.00261112
 0.00246049 0.00247845 0.00249571 0.00249785 0.00244328 0.00255591
 0.00248699 0.00251857 0.00247985 0.00248053 0.00240513 0.0025552
 0.00249902 0.00245256 0.00252311 0.00246118 0.00248712 0.00252399
 0.00248069 0.00251733 0.00246263 0.00248555 0.00236718 0.00254697
 0.00246518 0.00243368 0.00250015 0.00255309 0.00245062 0.00253862
 0.00248576 0.00251824 0.00240345 0.00247724 0.00249546 0.00245356
 0.00248564 0.00242592 0.00247821 0.00240704 0.00248448 0.00243738
 0.00249845 0.00256085 0.00260667 0.00257811 0.00252603 0.00246358
 0.00244035 0.00249852 0.00238015 0.00253373 0.00253589 0.00251

In [6]:
import torch
import torch.nn as nn

def init_weights(m):
    if isinstance(m, nn.Linear):
        torch.nn.init.xavier_uniform_(m.weight)
        m.bias.data.fill_(0.01)


class Feedforward(nn.Module):
  def __init__(self, input_size, hidden_size_1, hidden_size_2, hidden_size_3):
    super(Feedforward, self).__init__()

    self.mlp = nn.Sequential(nn.Linear(input_size, hidden_size_1), nn.ReLU(),
                             nn.Linear(hidden_size_1, hidden_size_2), nn.ReLU(),
                             nn.Linear(hidden_size_2,hidden_size_3), nn.ReLU(),
                             nn.Linear(hidden_size_3, 1), nn.Sigmoid())
    
    self.mlp.apply(init_weights)

  def forward(self, x):
    output = self.mlp(x.float())
    return output

In [7]:
import os
import random
from torch.utils.data import Dataset
from pandas import *


def generateDataset(root_dir, train, seed):
  
  #get labels into a list
  label_path = os.path.join(root_dir, "data50k_tweet.csv")
  data = read_csv(label_path)
  label_list = data['sentiment'].tolist()  

  #get tweets into a list
  tweet_path = os.path.join(root_dir, "data50k_tweet_wme_exp_0.npy")
  wme_mat = np.load(tweet_path)
  
  dataset = []
  for i in range(len(label_list)):
    dataset.append((label_list[i], wme_mat[i]))
  random.seed(seed)
  random.shuffle(dataset)


  if train:
    dataset = dataset[10000:]
  else:
    dataset = dataset[:10000]

  return dataset


class tweetDataset(Dataset):
  
  def __init__(self, root_dir, train=True, seed=0):
    self.root_dir = root_dir
    self.dataset = generateDataset(root_dir, train, seed)


  def __len__(self):
    return len(self.dataset)

  
  def __getitem__(self, idx):
    return self.dataset[idx]

In [8]:
my_dataset = tweetDataset("/content/drive/MyDrive/data") # training directory
my_loader = torch.utils.data.DataLoader(my_dataset, batch_size=50, shuffle=True, num_workers=0)

dataiter = iter(my_loader)
sentiment, tweet = dataiter.next()
print(sentiment)
print(tweet)

tensor([0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1,
        0, 0, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0,
        1, 0])
tensor([[0.0052, 0.0055, 0.0057,  ..., 0.0057, 0.0055, 0.0055],
        [0.0038, 0.0040, 0.0040,  ..., 0.0041, 0.0040, 0.0040],
        [0.0034, 0.0035, 0.0036,  ..., 0.0037, 0.0036, 0.0036],
        ...,
        [0.0053, 0.0055, 0.0056,  ..., 0.0057, 0.0055, 0.0055],
        [0.0033, 0.0033, 0.0034,  ..., 0.0035, 0.0034, 0.0033],
        [0.0043, 0.0046, 0.0047,  ..., 0.0047, 0.0047, 0.0045]],
       dtype=torch.float64)


In [9]:
# Device configuration
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [10]:
def save_model(model, optimizer, model_path, model_name):

    torch.save(model.state_dict(), '{}/{}/model_weights.pt'.format(model_path, model_name))
    torch.save(optimizer.state_dict(), '{}/{}/optimizer_weights.pt'.format(model_path, model_name))

def save_logs(epoch, loss, accuracy, type='train'):

    if type=='train':
      f = open('/content/drive/MyDrive/data/results_train.csv', 'a')
    elif type=='test':
      f = open('/content/drive/MyDrive/data/results_test.csv', 'a')
    for i in range(0, len(loss)):
        f.write('{},{},{}\n'.format(epoch, loss[i], accuracy[i]))
    f.close()

In [11]:
def accuracy_func(pred, target):
    rounded = torch.round(pred)
    # Get correct predictions
    accu = sum(rounded == target for rounded, target in zip(rounded, target)) / pred.shape[0]
    return accu

def train(model, loaders, criterion, optimizer, epochs):
    
    total_batches = len(loaders["train"]) * epochs
    test_loss_min = np.Inf
    batch_ct = 0
    sample_ct = 0
    
    for epoch in range(epochs):
        train_loss = 0.0
        test_loss = 0.0
        accuracy = 0.0

        # Train the model
        model.train()
        train_loss_list = []
        train_acc_list = []
        test_loss_list = []
        test_acc_list = []

        for batch_idx, (targets, inputs) in enumerate(loaders["train"]):
            # Send them to the GPU
            inputs, targets = inputs.to(device), targets.to(device)

            
            # optimizer.zero_grad()
            for param in model.parameters(): # Alternative recommended
                param.grad = None
            
            outputs = model(inputs)
            loss = criterion(outputs, targets.unsqueeze(1).type(torch.FloatTensor).to(device))
            loss.backward()
            optimizer.step()

            sample_ct += len(inputs)
            batch_ct += 1

            train_loss_list.append(loss.item())

            acc = accuracy_func(outputs, targets)
            train_acc_list.append(acc.item())

            train_loss += (1 / (batch_idx + 1)) * (loss.data - train_loss)

        #open file csv, for i in len(train_loss):
        # mettre dans le csv 
        save_logs(epoch, train_loss_list, train_acc_list, 'train')

        # Evaluate the model
        model.eval()
        with torch.no_grad():
            total = 0
            correct = 0
            for batch_idx, (targets, inputs) in enumerate(loaders["test"]):
                # Send them to the GPU
                inputs, targets = inputs.to(device), targets.to(device)
                total += targets.size(0)

                outputs = model(inputs)
                loss = criterion(outputs, targets.unsqueeze(1).to(torch.float))
                test_loss += (1 / (batch_idx + 1)) * (loss.data - test_loss)

                test_loss_list.append(loss.item())

                acc = accuracy_func(outputs, targets)
                test_acc_list.append(acc.item())

                predicted = torch.where(outputs > 0.5, 1, 0)
                correct += (predicted == targets.unsqueeze(1)).sum().item()

            save_logs(epoch, test_loss_list, test_acc_list, 'test')

        # Log it
        accuracy = 100 * correct / total
        #to_log = {"epoch": epoch,
        #         "training loss": train_loss,
        #         "testing loss": test_loss,
        #         "accuracy": accuracy,
        #         "time": time.time()-start}
        #wandb.log(to_log, step=sample_ct)
        print("Epoch: {} \tTraining Loss: {:.5f} \tTest Loss: {:.5f} \tAccuracy: {:.5f}%".format(epoch, train_loss, test_loss, accuracy))
        
        # Save the model
        if test_loss <= test_loss_min:
        #if False:
            print('Test loss decreased ({:.5f} --> {:.5f}). Saving model...\n'.format(test_loss_min, test_loss))
            #torch.onnx.export(model, inputs, "model.onnx")
            #wandb.save("model.onnx")
            test_loss_min = test_loss

    return model    

In [12]:
### First hyperparameters ###
num_workers = 6
batch_size = 8

# Loaders
training_dataset = tweetDataset("/content/drive/MyDrive/data", train=True)
testing_dataset = tweetDataset("/content/drive/MyDrive/data", train=False)

train_loader = torch.utils.data.DataLoader(training_dataset, batch_size=batch_size, shuffle=True,
                                           pin_memory=True, num_workers=num_workers)
test_loader = torch.utils.data.DataLoader(testing_dataset, batch_size=batch_size, shuffle=True,
                                          pin_memory=True, num_workers=num_workers)

loaders = {
    "train": train_loader,
    "test": test_loader
}

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 6 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [14]:
#momentum=0.005
learning_rate=1e-4

model = Feedforward(256, 128, 64, 32).to(device)
# Binary Cross Entropy Loss
criterion = nn.BCELoss()
#optimizer = torch.optim.Adam(model.parameters(), lr=config.learning_rate)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

print(model)
    
# Train it
model_f = train(model, loaders, criterion, optimizer, epochs=25)


Feedforward(
  (mlp): Sequential(
    (0): Linear(in_features=256, out_features=128, bias=True)
    (1): ReLU()
    (2): Linear(in_features=128, out_features=64, bias=True)
    (3): ReLU()
    (4): Linear(in_features=64, out_features=32, bias=True)
    (5): ReLU()
    (6): Linear(in_features=32, out_features=1, bias=True)
    (7): Sigmoid()
  )
)


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 6 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Epoch: 0 	Training Loss: 0.66754 	Test Loss: 0.65499 	Accuracy: 61.30000%
Test loss decreased (inf --> 0.65499). Saving model...

Epoch: 1 	Training Loss: 0.65411 	Test Loss: 0.65279 	Accuracy: 60.98000%
Test loss decreased (0.65499 --> 0.65279). Saving model...

Epoch: 2 	Training Loss: 0.65226 	Test Loss: 0.65345 	Accuracy: 61.36000%
Epoch: 3 	Training Loss: 0.65097 	Test Loss: 0.65058 	Accuracy: 61.43000%
Test loss decreased (0.65279 --> 0.65058). Saving model...

Epoch: 4 	Training Loss: 0.64939 	Test Loss: 0.64885 	Accuracy: 61.72000%
Test loss decreased (0.65058 --> 0.64885). Saving model...

Epoch: 5 	Training Loss: 0.64812 	Test Loss: 0.64838 	Accuracy: 61.40000%
Test loss decreased (0.64885 --> 0.64838). Saving model...

Epoch: 6 	Training Loss: 0.64752 	Test Loss: 0.64989 	Accuracy: 61.54000%
Epoch: 7 	Training Loss: 0.64582 	Test Loss: 0.66377 	Accuracy: 60.33000%
Epoch: 8 	Training Loss: 0.64437 	Test Loss: 0.65108 	Accuracy: 60.80000%
Epoch: 9 	Training Loss: 0.64251 	Test

### This had to be run on our PC because Colab doesn't accept to plot and save the figures

In [34]:
import numpy as np
import matplotlib.pyplot as plt

def get_epoch_idx(seq):

    epoch_idx = []
    start_epoch = 0
    for i in range(len(seq)):
        if int(seq[i]) > start_epoch:
            epoch_idx.append(i)
            start_epoch = int(seq[i])

    return epoch_idx


def avg_smoothing(seq, window_size):

    start_idx = 0
    end_idx = window_size
    target_idx = int(window_size / 2)
    output = seq.copy()

    while end_idx < len(seq):
        output[target_idx] = np.mean(seq[start_idx:end_idx])
        start_idx += 1
        end_idx += 1
        target_idx += 1

    return output


idx = 0
epoch_idx = []

#parameters
smoothing_widow = 1000
alf_window = int(smoothing_widow / 2)
column_names = ["Epoch", "loss", "accuracy"]
red = 'tab:red'
blue = 'tab:blue'

#read CSV files
train_logs = pd.read_csv(
  './results_train.csv',
  header=None,
  names = column_names,
  index_col=False,
  encoding='latin')

#problem when reading first row of first column
train_logs['Epoch'] = train_logs['Epoch'].replace(['ï»¿0'],'0')

#to plot 2 variables on same graph
fig, ax1 = plt.subplots()
ax2 = ax1.twinx()  # instantiate a second axes that shares the same x-axis

#first variable 
loss = train_logs['loss'].to_numpy()
loss = avg_smoothing(loss, smoothing_widow)
loss = loss[alf_window:-alf_window]

ax1.plot(np.arange(len(loss)), loss,
                    label ="Train loss",
                    linewidth=0.3,
                    color=red)
ax1.set_ylabel('Loss')


#second variable
acc = train_logs['accuracy'].to_numpy()
acc = avg_smoothing(acc, smoothing_widow)
acc = acc[alf_window:-alf_window]

ax2.plot(np.arange(len(acc)), acc,
                     label="Train accuracy",
                     linewidth=0.3,
                     color=blue)
ax2.set_ylabel('Accuracy')


#get epochs
epoch_idx = get_epoch_idx(train_logs['Epoch'].to_numpy())

[plt.axvline(x=epoch_idx[i], color='black',linewidth=0.2, linestyle='-')
     for i in range(0, len(epoch_idx))]

lines, labels = ax1.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax2.legend(lines + lines2, labels + labels2, loc='center right')

plt.title('Training logs')
ax1.set_xlabel('Batch')
plt.savefig('./train_perfs.png', format='png')
plt.show()

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def get_epoch_idx(seq):

    epoch_idx = []
    start_epoch = 0
    for i in range(len(seq)):
        if int(seq[i]) > start_epoch:
            epoch_idx.append(i)
            start_epoch = int(seq[i])

    return epoch_idx


def avg_smoothing(seq, window_size):

    start_idx = 0
    end_idx = window_size
    target_idx = int(window_size / 2)
    output = seq.copy()

    while end_idx < len(seq):
        output[target_idx] = np.mean(seq[start_idx:end_idx])
        start_idx += 1
        end_idx += 1
        target_idx += 1

    return output


idx = 0
epoch_idx = []

#parameters
smoothing_widow = 200
alf_window = int(smoothing_widow / 2)
column_names = ["Epoch", "loss", "accuracy"]
red = 'tab:red'
blue = 'tab:blue'

#read CSV files
test_logs = pd.read_csv(
  '/content/drive/MyDrive/data/results_test.csv',
  header=None,
  names = column_names,
  index_col=False,
  encoding='latin')

#problem when reading first row of first column
test_logs['Epoch'] = test_logs['Epoch'].replace(['ï»¿0'],'0')




#to plot 2 variables on same graph
fig, ax1 = plt.subplots()
ax2 = ax1.twinx()  # instantiate a second axes that shares the same x-axis

#first variable 
loss = test_logs['loss'].to_numpy()
loss = avg_smoothing(loss, smoothing_widow)
loss = loss[alf_window:-alf_window]

ax1.plot(np.arange(len(loss)), loss,
                    label ="Test loss",
                    linewidth=0.3,
                    color=red)
ax1.set_ylabel('Loss')


#second variable
acc = test_logs['accuracy'].to_numpy()
acc = avg_smoothing(acc, smoothing_widow)
acc = acc[alf_window:-alf_window]

ax2.plot(np.arange(len(acc)), acc,
                     label="Test accuracy",
                     linewidth=0.3,
                     color=blue)
ax2.set_ylabel('Accuracy')


#get epochs
epoch_idx = get_epoch_idx(test_logs['Epoch'].to_numpy())

[plt.axvline(x=epoch_idx[i], color='black',linewidth=0.2, linestyle='-')
     for i in range(0, len(epoch_idx))]

lines, labels = ax1.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax2.legend(lines + lines2, labels + labels2, loc='center right')

plt.title('Test logs')
ax1.set_xlabel('Batch')
plt.savefig('/content/drive/MyDrive/data/test_perfs.png', format='png')
plt.show()